****STDL PROJECT NOTEBOOK****

**Phase 1: predict gene expression levels of one specific gene over given input biopsy images**

Assign GPU device

In [1]:
import torch
print(f'cuda device count: {torch.cuda.device_count()}')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
#Additional Info when using cuda
if device.type == 'cuda':
    print(f'device name: {torch.cuda.get_device_name(0)}')
    print(f'torch.cuda.device(0): {torch.cuda.device(0)}')
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_cached(0)/1024**3,1), 'GB')
# NOTE: important !!!!!!
# clearing out the cache before beginning
torch.cuda.empty_cache()


cuda device count: 1
Using device: cuda
device name: GeForce RTX 2080 Ti
torch.cuda.device(0): <torch.cuda.device object at 0x7f7fac644950>
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB



Import project files

In [2]:
# create code to reimport module if i change it
%load_ext autoreload
%autoreload 2

# note: path to project is: /home/roy.rubin/STDLproject/
import loadAndPreProcess
import deepNetworkArchitechture

perform pre processing actions

In [3]:
print("\nHi! welcome to the program :)\n")


Hi! welcome to the program :)



In [4]:
path_to_images_dir = "/home/roy.rubin/STDLproject/spatialGeneExpressionData/images"
imageFolder = loadAndPreProcess.load_dataset_from_images_folder(path_to_images_dir)


----- entered function load_dataset_from_pictures_folder -----


Do you wish to print information about the ImageFolder dataset object ? [yes/no] n



----- finished function load_dataset_from_pictures_folder -----



In [5]:
path_to_mtx_tsv_files_dir = "/home/roy.rubin/STDLproject/spatialGeneExpressionData"
matrix_dataframe, features_dataframe, barcodes_datafame = loadAndPreProcess.load_dataframes_from_mtx_and_tsv_new(path_to_mtx_tsv_files_dir)


----- entered function load_dataframes_from_mtx_and_tsv -----
started reading features.tsv
V  finished reading features.tsv
started reading barcodes.tsv
V  finished reading barcodes.tsv
started reading matrix.mtx. this might take some time ...
V  finished reading matrix.mtx


Do you wish to print information about the 3 loaded dataframes ? [yes/no] n



----- finished function load_dataframes_from_mtx_and_tsv -----



remove empty genes (i think there are only 3 of these ... need to verify)

In [7]:
%autoreload 2
matrix_dataframe, mapping_between_old_and_new_indices = loadAndPreProcess.cut_empty_genes(matrix_dataframe)  #TODO: uncomment later

Do you wish to print information about the reduced dataframe ? [yes/no] y



print data regarding the reduced dataframe:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23073 entries, 0 to 23072
Columns: 4992 entries, 0 to 4991
dtypes: Sparse[int64, 0](4992)
memory usage: 245.2 MB
None
   0     1     2     3     4     5     6     7     8     9     ...  4982  \
0     0     0     0     0     0     0     0     0     0     1  ...     0   
1     0     0     0     0     0     0     0     0     0     0  ...     0   
2     0     0     0     0     0     0     0     0     0     0  ...     0   
3     0     0     0     2     0     0     0     0     0     0  ...     0   
4     0     0     0     0     0     0     0     0     0     0  ...     0   

   4983  4984  4985  4986  4987  4988  4989  4990  4991  
0     0     0     0     0     0     0     0     0     0  
1     0     0     1     0     0     0     0     0     0  
2     0     0     0     0     0     0     0     0     0  
3     0     0     0     0     0     0     0     0     0  
4     0     0     0     0     0     0   

In [8]:
print(f'\note: this is the mapping_between_old_and_new_indices: \n{mapping_between_old_and_new_indices}')


verify: mapping_between_old_and_new_indices        original_index_from_matrix_dataframe
0                                         3
1                                         4
2                                        11
3                                        12
4                                        13
...                                     ...
23068                                 33528
23069                                 33530
23070                                 33532
23071                                 33533
23072                                 33535

[23073 rows x 1 columns]


In [9]:
gene_name = 'MKI67'
stdl_ds1 = loadAndPreProcess.STDL_Dataset_SingleValuePerImg(imageFolder=imageFolder, 
                                                               matrix_dataframe=matrix_dataframe, 
                                                               features_dataframe=features_dataframe, 
                                                               barcodes_datafame=barcodes_datafame, 
                                                               chosen_gene_name=gene_name)


----- entering __init__ phase of  STDL_Dataset -----

----- finished __init__ phase of  STDL_Dataset -----



In [10]:
k = 10
stdl_ds2 = loadAndPreProcess.STDL_Dataset_KValuesPerImg_KGenesWithHighestVariance(imageFolder=imageFolder, 
                                                                           matrix_dataframe=matrix_dataframe, 
                                                                           features_dataframe=features_dataframe, 
                                                                           barcodes_datafame=barcodes_datafame, 
                                                                           num_of_dims_k=k)


----- entering __init__ phase of  STDL_Dataset_KValuesPerImg_KGenesWithHighestVariance -----
calculate variance of all columns from  matrix_dataframe - and choosing K genes with higest variance ...

----- finished __init__ phase of  STDL_Dataset_LatentTensor -----



In [11]:
k = 10
stdl_ds3 = loadAndPreProcess.STDL_Dataset_KValuesPerImg_LatentTensor_NMF(imageFolder=imageFolder, 
                                                                           matrix_dataframe=matrix_dataframe, 
                                                                           features_dataframe=features_dataframe, 
                                                                           barcodes_datafame=barcodes_datafame, 
                                                                           num_of_dims_k=k)


----- entering __init__ phase of  STDL_Dataset_KValuesPerImg_LatentTensor_NMF -----
performing NMF decomposition on main matrix dataframe ...
--delete--self.reduced_dataframe type <class 'numpy.ndarray'>
--delete--self.reduced_dataframe  [[4.98120439e-05 1.33850363e-02 4.76250707e-02 ... 1.03459070e-01
  3.27783168e-01 0.00000000e+00]
 [4.02429736e-03 3.36941569e-01 2.56214584e-02 ... 3.61699063e-01
  4.35347928e-02 4.77425038e-02]
 [0.00000000e+00 0.00000000e+00 1.72022291e-01 ... 1.74486023e-01
  0.00000000e+00 1.29654143e-01]
 ...
 [2.44588111e-02 4.66592345e-01 4.96592325e-02 ... 0.00000000e+00
  7.66718958e-01 2.30755473e-01]
 [8.09784976e-02 3.82302882e-01 7.11604566e-01 ... 4.63555790e-01
  1.20645204e+00 1.79719735e+00]
 [0.00000000e+00 3.56613390e-01 3.13505375e-02 ... 2.69741285e-01
  0.00000000e+00 1.62540193e-02]]
--delete---after- self.reduced_dataframe type <class 'pandas.core.frame.DataFrame'>
--delete---after- self.reduced_dataframe         0         1         2       

In [12]:
%autoreload 2

# print('Memory Cached so far on Cuda:   ', round(torch.cuda.memory_cached(0)/1024**3,1), 'GB')
# print('Out of the Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
# # NOTE: important !!!!!!
# # clearing out the cache before beginning
# torch.cuda.empty_cache()
# print('Memory Cached so far on Cuda:   ', round(torch.cuda.memory_cached(0)/1024**3,1), 'GB')
# print('Out of the Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')

k = 10
stdl_ds4 = loadAndPreProcess.STDL_Dataset_KValuesPerImg_LatentTensor_AutoEncoder(imageFolder=imageFolder, 
                                                                           matrix_dataframe=matrix_dataframe, 
                                                                           features_dataframe=features_dataframe, 
                                                                           barcodes_datafame=barcodes_datafame, 
                                                                           num_of_dims_k=k,
                                                                           device=device)


----- entering __init__ phase of  STDL_Dataset_KValuesPerImg_LatentTensor_AutoEncoder -----
initializing the autoencoder (this might take a while) ...

----- entered function return_trained_AE_net -----
--delete-- verify inside STDL_Dataset_matrix_df_for_AE_init __init__: self.data.shape: (23073, 4992)
note - number of (hidden) linear layers is supposed to be 3

entered __init__ of AutoencoderNet
****** begin training ******

iteration 1 of 3 epochs
batch 2000 of 2000 batches
finished inner loop.

in this epoch: min loss 0.0040384624153375626 max loss 778.905029296875
               average loss 10.268329235401703

iteration 2 of 3 epochs
batch 2000 of 2000 batches
finished inner loop.

in this epoch: min loss 0.003464254317805171 max loss 783.9368286132812
               average loss 10.482129897606443

iteration 3 of 3 epochs
batch 2000 of 2000 batches
finished inner loop.

in this epoch: min loss 0.004021810367703438 max loss 580.6859130859375
               average loss 7.56304612


display a few sample images

In [13]:
# TODO: code to display some sample images
# 

check sizes of a pair (x,y) from all different datasets

In [15]:
print(f'stdl_ds1 (single gene):      x shape {stdl_ds1[0][0].shape}, y is an int')
print("---------")
print(f'stdl_ds2 (highest variance): x shape {stdl_ds2[0][0].shape}, y shape {stdl_ds2[0][1].shape}')
print("---------")
print(f'stdl_ds3 (NMF):              x shape {stdl_ds3[0][0].shape}, y shape {stdl_ds3[0][1].shape}')
print("---------")
print(f'stdl_ds4 (AE):               x shape {stdl_ds4[0][0].shape}, y shape {stdl_ds4[0][1].shape}')


stdl_ds1: x shape torch.Size([3, 176, 176]), y shape 0
---------
stdl_ds2: x shape torch.Size([3, 176, 176]), y shape (10,)
---------
stdl_ds3: x shape torch.Size([3, 176, 176]), y shape (10,)
---------
stdl_ds4: x shape torch.Size([3, 176, 176]), y shape torch.Size([10])


In [ ]:
%load_ext autoreload
%autoreload 2
import executionModule

# executionModule.runTest1_singleGenePrediction(stdl_ds, device)  #uncomment later
# executionModule.runTest2_allGenePrediction_dimReduction_KHighestVariances(stdl_ds2, device)
# executionModule.runTest3_allGenePrediction_dimReduction_NMF(stdl_ds2, device)
# executionModule.runTest4_allGenePrediction_dimReduction_AutoEncoder(stdl_ds4, device)